In [1]:
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate

# For caching models and logging
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 2.2 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 7.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 4.8 MB/s eta 0:00:00:00:0100:01

In [2]:
import torch

# Check GPU status
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Available Memory: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")

# Clear cache
torch.cuda.empty_cache()

GPU: Tesla T4
Total Memory: 15.83 GB
Available Memory: 15.83 GB


In [ ]:
import os
from huggingface_hub import login

# Load token from environment variable
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

In [4]:
import os
os.environ['VLLM_USE_V1'] = '0'

import torch
import gc

torch.cuda.empty_cache()
gc.collect()

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()

import pandas as pd
from vllm import LLM, SamplingParams
from tqdm import tqdm

# ডাটা লও
df = pd.read_csv('/kaggle/input/banglachq/train.csv')
df = df[['id', 'question']].copy()
print(f"লোড অইল {len(df)}টা লাইন।")

model_name = "google/gemma-3-1b-it"
print(f"\nমডেল লোড অইতাসে {model_name}...")

llm = LLM(
    model=model_name,
    trust_remote_code=True,
    dtype="float32",  # 1B should work with float16 on T4
    gpu_memory_utilization=0.7,
    max_model_len=2048,
    max_num_seqs=24,
)

tokenizer = llm.get_tokenizer()
print("মডেল লোড অইসে!\n")

# প্রম্পট আগের মতোই রাইখা
def translate_to_chatgaiyan(bangla_text):
    prompt = f"""You are a precise translation tool. Your task is to translate the given Bangla text to Chittagonian (Chatgaiyan) dialect using Bengali script.

        INSTRUCTIONS:
        - Analyze the text step-by-step before translating
        - Use only Bengali script in your final translation
        - After your reasoning, provide ONLY the final translation with no additional commentary

        Here are examples showing the translation process:

        Example 1:
        Bangla: খালু, গ্রামের বাড়ি যাবেন কবে
        Reasoning: "খালু" becomes "অহালু" (phonetic shift with অ prefix), "গ্রামের বাড়ি" becomes "গ্রামর বাড়িত" (possessive -এর→-র, locative case -ত added), "যাবেন কবে" becomes "হত্তে যাইবেন?" (emphatic হত্তে added, verb form changed, question marker)
        Chittagonian: অহালু, গ্রামর বাড়িত হত্তে যাইবেন?

        Example 2:
        Bangla: আপনি কি নিয়মিত জল খান?
        Reasoning: "আপনি" becomes "অনে" (phonetic compression), "কি নিয়মিত" becomes "কি পইত্তেন" (restructured with পইত্তেন for regularity concept), "জল খান" becomes "পানি হন না?" (জল→পানি, খান→হন না with negative question form)
        Chittagonian: অনে কি পইত্তেন পানি হন না?

        Example 3:
        Bangla: জুম্মার পর যাব।
        Reasoning: "জুম্মার" becomes "জুমর" (possessive -আর→-র shortened), "পর" stays as "পর" (unchanged), "যাব" becomes "যাইয়ুম" (first person future tense -ব→-ইয়ুম)
        Chittagonian: জুমর পর যাইয়ুম।

        Example 4:
        Bangla: পালং শাক খুব পছন্দ
        Reasoning: "পালং শাক" stays as "পালংশাক" (compound unchanged), "খুব" becomes "বেশি" (lexical replacement), "পছন্দ" becomes "গম লাগে দে" (idiomatic expression with গম=liking, লাগে=feels, দে=emphasis particle)
        Chittagonian: পালংশাক বেশি গম লাগে দে

        Example 5:
        Bangla: বাঙালি নানা-নানীরা কি করেন?
        Reasoning: "বাঙালি" becomes "বাআলর" (phonetic compression with possessive -র), "নানা-নানী-রা" becomes "নানানানী অলে" (compound simplified, plural -রা→অলে), "কি করেন" becomes "কি গরে দে?" (করেন→গরে with দে emphasis particle)
        Chittagonian: বাআলর নানানানী অলে কি গরে দে?

        Now translate this text:
        Bangla: {bangla_text}

        Think through the translation:
        - Identify key vocabulary transformations
        - Note grammatical changes (possessives, verb forms, cases)
        - Apply Chittagonian phonetic and structural patterns

        Provide your final translation below:
        Chittagonian:""" 
    
    messages = [{"role": "user", "content": prompt}]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

sampling_params = SamplingParams(
    temperature=0.3,
    top_p=0.9,
    max_tokens=512,
    stop=["Bangla:"]
)

BATCH_SIZE = 24
all_chatgaiyan = []

for batch_start in tqdm(range(0, len(df), BATCH_SIZE), desc="অনুবাদ করতাছে"):
    batch_end = min(batch_start + BATCH_SIZE, len(df))
    batch_prompts = [translate_to_chatgaiyan(row['question']) for _, row in df.iloc[batch_start:batch_end].iterrows()]
    
    outputs = llm.generate(batch_prompts, sampling_params)
    all_chatgaiyan.extend([o.outputs[0].text.strip() for o in outputs])
    
    del outputs, batch_prompts
    gc.collect()
    torch.cuda.empty_cache()

df['chatgaiyan_translation'] = all_chatgaiyan
df.to_csv('gemma_0_shot_chatgaiyan.csv', index=False)
print("হইল!")


2025-09-28 18:55:38.543298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759085738.766711      60 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759085738.832932      60 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 09-28 18:55:53 [__init__.py:216] Automatically detected platform cuda.
লোড অইল 1880টা লাইন।

মডেল লোড অইতাসে google/gemma-3-1b-it...
INFO 09-28 18:55:55 [utils.py:328] non-default args: {'trust_remote_code': True, 'dtype': 'float32', 'max_model_len': 2048, 'gpu_memory_utilization': 0.7, 'max_num_seqs': 24, 'disable_log_stats': True, 'model': 'google/gemma-3-1b-it'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

INFO 09-28 18:56:11 [__init__.py:742] Resolved architecture: Gemma3ForCausalLM


...

অনুবাদ করতাছে:  99%|█████████▊| 78/79 [25:45<00:22, 22.33s/it]

Adding requests:   0%|          | 0/8 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

অনুবাদ করতাছে: 100%|██████████| 79/79 [26:01<00:00, 19.77s/it]

হইল!
